## LangChain 使用（基于DeepSeek）

### 0.安装LangChain相关包 （只需要执行一次）

In [ ]:
!pip install langchain

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     ------------------------------------ --- 1.8/2.0 MB 9.5 MB/s eta 0:00:01
     ------------------------------------ --- 1.8/2.0 MB 9.5 MB/s eta 0:00:01
     ------------------------------------ --- 1.8/2.0 MB 9.5 MB/s eta 0:00:01
     ------------------------------------ --- 1.8/2.0 MB 9.5 MB/s eta 0:00:01
     ------------------------------------ --- 1.8/2.0 MB 9.5 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 1.5 MB/s  0:00:01

   --- ------------------------------------  3/31 [urllib3]
   --- ------------------------------------  3/31 [urllib3]
   --- ------------------------------------  3/31 [urllib3]
   --- ------------------------------------  3/31 [urllib3]
   --- ------------------------------------  3/31 [urllib3]
   ------ ---------------------------------  5/31 [tenacity]
   ------- -------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install langchain_community

In [3]:
!pip install langchain-deepseek 

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 1.0/1.0 MB 6.3 MB/s  0:00:00
     ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
     ---------------------------------------- 879.1/879.1 kB 6.2 MB/s  0:00:00

   ---------------------------------------- 0/9 [tqdm]
   ---------------------------------------- 0/9 [tqdm]
   ---------------------------------------- 0/9 [tqdm]
   ---- ----------------------------------- 1/9 [sniffio]
   -------- ------------------------------- 2/9 [regex]
   ----------------- ---------------------- 4/9 [distro]
   ---------------------- ----------------- 5/9 [tiktoken]
   -------------------------- ------------- 6/9 [openai]
   -------------------------- ------------- 6/9 [openai]
   -------------------------- ------------- 6/9 [openai]
   -------------------------- ------------- 6/9 [openai]
   ---------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 指定Key

In [2]:
import os
os.environ["DEEPSEEK_API_BASE"] = 'https://api.deepseek.com/v1'
os.environ["DEEPSEEK_API_KEY"] = 'sk-60e9910f323c46c3aeb645951b2d45ec'

### 定义模型

In [3]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### 执行问答

In [6]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to Chinese."),
    ("human", "I love programming."),
]
model.invoke(messages)

AIMessage(content='我喜欢编程。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 21}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '99d74395-2e26-41a1-a164-d442684a0973', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0b66-018f-7113-b1f7-75c722b9f2de-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 1. LangChain消息不同角色

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="将以下内容翻译为英文"),
    HumanMessage(content="你好"),
]

response = model.invoke(messages)

In [8]:
print(response.content)

Hello


In [9]:
print(response)

content='Hello' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3fb48d09-448e-4391-b43a-8b85babb6fb0', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b0b66-19c8-7d80-9a4a-c74a0a100b09-0' usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [10]:
type(response)

langchain_core.messages.ai.AIMessage

### 2. LangChain中输出处理

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [12]:
parser.invoke(response)

'Hello'

### 3. LangChain 链

In [13]:
chain = model | parser

In [14]:
chain.invoke(messages)

'Hello'

### 4. LangChain中prompt模板

In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [16]:
system_template = "将以下内容翻译为{language}:"

In [17]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [18]:
result = prompt_template.invoke({"language": "意大利语", "text": "你好"})

print(result)

messages=[SystemMessage(content='将以下内容翻译为意大利语:', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={})]


In [19]:
result.to_messages()

[SystemMessage(content='将以下内容翻译为意大利语:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好', additional_kwargs={}, response_metadata={})]

In [20]:
chain = prompt_template | model | parser

In [21]:
chain.invoke({"language": "英文", "text": "你好"})

'Hello'

In [22]:
chain.invoke({"language": "法语", "text": "你好"})

'Bonjour'

In [23]:
chain.invoke({"language": "意大利语", "text": "你好"})

'Ciao'

## 5. 基于LangChain构建Chatbot

In [24]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好！我是Leo")])

AIMessage(content='你好，Leo！很高兴认识你！😊\n\n我是DeepSeek，一个由深度求索公司创造的AI助手。无论你有什么问题、需要什么帮助，或者只是想聊聊天，我都很乐意为你提供支持。\n\n我可以帮你处理各种任务，比如：\n- 回答问题和解释概念\n- 协助写作和编辑\n- 分析和处理文档\n- 编程和技术支持\n- 创意 brainstorming\n- 学习辅导等等\n\n有什么我可以为你做的吗？或者你想聊聊什么话题呢？我很期待和你的交流！✨', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 8, 'total_tokens': 119, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '698b95ba-0b73-4082-9da3-35c87c9ad905', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0b68-5db6-76c1-8760-8d7d294d712d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 111, 'total_tokens': 119, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 但是模型不具有记忆

In [25]:
model.invoke([HumanMessage(content="我叫什么名字?")])

AIMessage(content='我无法知道您的名字哦！😊 作为AI助手，我没有访问个人信息的能力，也不知道您是谁。\n\n如果您愿意的话，可以告诉我您希望我怎么称呼您，这样我们的对话会更亲切一些！或者您也可以保持匿名，这完全没问题。\n\n有什么其他我可以帮助您的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 8, 'total_tokens': 67, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'bbb87b12-b5c9-4030-8929-f5a78953d871', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0b68-7cd9-7300-9205-f5ad156e87d6-0', usage_metadata={'input_tokens': 8, 'output_tokens': 59, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 结合上下文对话，可以输出正确结果

In [26]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好！我是Leo"),
        AIMessage(content="你好，Leo！很高兴见到你。有什么我可以帮助你的吗？"),
        HumanMessage(content="我叫什么名字?"),
    ]
)

AIMessage(content='你刚才提到你的名字是 **Leo**。如果这不是你希望使用的名字，或者需要我更改称呼，请随时告诉我！ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 30, 'total_tokens': 59, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 30}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '75ae5aba-148b-451b-98b1-07a62d369e42', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0b68-9640-7502-a42e-668945f82762-0', usage_metadata={'input_tokens': 30, 'output_tokens': 29, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 消息历史

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [29]:
config = {"configurable": {"session_id": "abc1"}}

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="你好！我是Leo")],
    config=config,
)

response.content

'你好，Leo！很高兴认识你！😊  \n我是DeepSeek，一个AI助手，由深度求索公司创造。无论你想聊天、寻求帮助，还是需要解答问题，我都会尽力为你提供支持。  \n有什么我可以帮你的吗？'

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="我叫什么名字?")],
    config=config,
)

response.content

'你刚才告诉我，你的名字是 **Leo**！😊  \n如果这不是你真实的名字，或者你想让我用其他名字称呼你，请随时告诉我～'

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="我有一条宠物狗，它的名字是旺财")],
    config=config,
)

response.content

'哇，旺财！这个名字听起来特别有福气，而且在中国文化里寓意着“招财进宝”呢～🐶  \n你的旺财是什么品种的狗狗呀？或者可以分享一下它的趣事吗？我超喜欢听关于宠物的故事！✨'

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="我的宠物叫什么?")],
    config=config,
)

response.content

'你刚刚告诉我，你的宠物狗名字叫 **旺财**～🐕  \n（如果它有小名或者我记错了，记得提醒我哦！）😊'

### 流式输出

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("告诉我一个关于 {topic} 的故事")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "猫"}):
    print(chunk, end="", flush=True)

# 猫的报恩

在东京一条安静的巷子里，住着一位独居的老画家，名叫佐藤。他性格孤僻，唯一的伴侣是一只名叫“墨”的黑猫。墨是佐藤在雨夜捡回来的流浪猫，右耳缺了一角，琥珀色的眼睛总带着警惕。

佐藤的生活规律而单调：早晨画画，下午散步，晚上读书。墨则安静地陪在他身边，偶尔跳上画架，在未干的油画上留下梅花般的爪印。佐藤从不生气，只是笑着擦掉，重新画过。

一个冬夜，佐藤心脏病突发，倒在了画室地板上。墨焦急地绕着他转圈，用头蹭他的脸，用爪子拍打他的手臂，但佐藤毫无反应。

墨跳上窗台，望着外面飘雪的黑夜，琥珀色的眼睛闪过一丝决绝。它用头撞开窗户的插销，纵身跃入风雪中。

巷子尽头有一家24小时便利店，店员山田正在打瞌睡。突然，一只湿漉漉的黑猫冲进店里，对着他大声叫唤。

“去去，这里不能进来。”山田挥手驱赶。

墨不退反进，咬住山田的裤脚往外拖。山田觉得奇怪，跟着墨走出店门。墨跑几步就回头看他，确保他跟着。

就这样，墨领着山田穿过小巷，来到佐藤家敞开的窗前。山田探头一看，大惊失色，立刻打电话叫了救护车。

佐藤被及时送往医院，捡回一条命。医生对山田说：“再晚半小时就危险了。你怎么知道他在家发病？”

山田指着蹲在急诊室门口的墨：“是这只猫带我去的。”

消息传开后，邻居们都称墨为“灵猫”。但墨并不在意这些，它每天穿过半个城市，准时出现在医院病房的窗台上，隔着玻璃陪伴佐藤。

一个月后，佐藤出院回家。他抱着墨，老泪纵横：“是你救了我，墨。”

墨只是轻轻蹭了蹭他的脸，跳上画架，在佐藤新铺的画布上，印下一个完美的梅花爪印。

从那天起，佐藤的画风变了。他不再画孤独的风景，而是画猫——各种各样的猫，在阳光下嬉戏，在屋檐上打盹，在樱花树下漫步。他的画作充满了温暖与生机，很快在艺术界引起了轰动。

一位著名评论家写道：“佐藤晚年的作品，仿佛被某种温柔的灵魂触碰过。”

只有佐藤知道，那个灵魂此刻正蜷缩在他的膝上，发出满足的呼噜声。

几年后的春天，佐藤在睡梦中安详离世。邻居发现时，墨静静卧在他身边，头枕着他的手心。

人们以为墨会再次流浪，但它没有离开。它依然住在佐藤的房子里，由邻居轮流照顾。奇怪的是，每年佐藤的忌日，墨都会消失一整天。

跟踪它的人发现，墨会去墓园，在佐藤的墓碑前放下一件小东西——有时是一片漂亮的羽毛，有时是一颗光滑的石头，有时是一朵野花。

第七年春天，墨没有从